# Demo 0: KX2 & SCILA & Byonoy Routine

## Import Statemets

In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
import asyncio
from tqdm.auto import tqdm

---
## SCILA Incubator Setup

1. import modules you require
2. create a SCILA incubator backend, and give it the device's IP address
3. call `.setup()` to establish a connection and initialize the device
4. 

In [3]:
from pylabrobot.storage.inheco.scila import SCILABackend

scila = SCILABackend(scila_ip="169.254.1.117")

await scila.setup()

### Status Requests

In [4]:
await scila.request_status()

'idle'

In [5]:
await scila.request_drawer_statuses()

{1: 'Closed', 2: 'Closed', 3: 'Closed', 4: 'Closed'}

In [6]:
drawer_indices = list(range(1, 5))

for idx in drawer_indices:
    
    temp_reponse = await scila.request_drawer_status(idx)

    print(idx, temp_reponse)

1 Closed
2 Closed
3 Closed
4 Closed


### Drawer Control

Note: By firmware, only one drawer can be opened at a time!

In [33]:
await scila.open(2)

In [32]:
await scila.close(1)

In [9]:
for idx in tqdm(drawer_indices):
    
    await scila.open(idx)

    await asyncio.sleep(1) # wait for 1 second
    
    await scila.close(idx)


  0%|          | 0/4 [00:00<?, ?it/s]

### Temperature Control

In [10]:
await scila.start_temperature_control(37.0)

In [12]:
for time_step in tqdm(range(10)):
    
    temperature_reading = await scila.measure_temperature()

    print(temperature_reading)

    await asyncio.sleep(1)

  0%|          | 0/10 [00:00<?, ?it/s]

25.72
25.97
26.19
26.4
26.6
26.8
27.0
27.2
27.37
27.56


Uncomment the next coding cell if you want to stop temperature control:

In [ ]:
# await scila.deactivate_temperature_control()

---
## KX2 Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pylabrobot.arms.kx2.kx2_backend import *
from copy import deepcopy

In [3]:
kx2 = KX2Backend()
can = kx2.can

In [4]:
x = await kx2.initialize()

motor send command 1 UI 4  False (1)
motor send command 2 UI 4  False (1)
motor send command 3 UI 4  False (1)
motor send command 4 UI 4  False (1)
motor send command 6 UI 4  False (1)

axis 1
motor send command 1 UI 5  False (1)
motor send command 1 UI 6  False (1)
motor send command 1 UI 7  False (1)
motor send command 1 UI 8  False (1)
motor send command 1 UI 9  False (1)
motor send command 1 UI 10  False (1)
motor send command 1 UI 11  False (1)
motor send command 1 UI 12  False (1)
motor send command 1 UI 13  False (1)
motor send command 1 UI 14  False (1)
motor send command 1 UI 15  False (1)
motor send command 1 UI 16  False (1)
motor send command 1 UI 24  False (1)
motor send command 1 UF 1  True (2)
motor send command 1 UF 2  True (2)
motor send command 1 XM 1  False (1)
motor send command 1 XM 2  False (1)
motor send command 1 UF 3  True (2)
motor send command 1 UF 4  True (2)
motor send command 1 VH 3  False (1)
motor send command 1 VL 3  False (1)
motor send command 1 CA 45

In [5]:
home_pose = GripperPose(
    location=Coordinate(x=-56.3944, y=154.9451, z=693.4449),
    rotation=Rotation(x=0, y=0, z=-167.0096158980516)
)
await kx2.move_to_cartesian_position(
    home_pose,
    vel_pct=20,
    accel_pct=20
)

cmd {<KX2Axis.SHOULDER: 1>: 19.999672055449714, <KX2Axis.Z: 2>: 693.4449, <KX2Axis.ELBOW: 3>: 13.232789640420236, <KX2Axis.WRIST: 4>: 172.9907120464987}


### Definition Routine

In [5]:
teaching_mode = True

In [6]:
if teaching_mode:
    await kx2.activate_free_mode()

In [7]:
if teaching_mode:
    await kx2.deactivate_free_mode()

### Posture Definitions

In [8]:
idle_location = await kx2.get_cartesian_position()

idle_location = GripperPose(
    location=Coordinate(x=6.282, y=434.0819, z=410.0812),
    rotation=Rotation(x=0, y=0, z=196.06784840324508)
)

In [9]:
byonoy_reader_location = GripperPose(
    location=Coordinate(x=-201.0336, y=519.1706, z=37.7131),
    rotation=Rotation(x=0, y=0, z=-162.9854014319016)
)


In [10]:
byonoy_parking_location = await kx2.get_cartesian_position()

byonoy_parking_location = GripperPose(
    location=Coordinate(x=8.4036, y=576.9749, z=37.5711),
    rotation=Rotation(x=0, y=0, z=197.05129643307828)
)

In [11]:
plate_on_byonoy_reader_location = await kx2.get_cartesian_position()

plate_on_byonoy_reader_location = GripperPose(
    location=Coordinate(x=-198.4042, y=514.5388, z=19.831),
    rotation=Rotation(x=0, y=0, z=-162.5829842296718)
)

In [12]:
plate_in_pico_location = await kx2.get_cartesian_position()

plate_in_pico_location = GripperPose(
    location=Coordinate(x=289.8992, y=581.4979, z=223.1917),
    rotation=Rotation(x=0, y=0, z=201.42433321429007)
)

In [13]:
plate_in_scilar_drawer_2_location = await kx2.get_cartesian_position()

plate_in_scilar_drawer_2_location = GripperPose(
    location=Coordinate(x=99.475, y=308.8323, z=38.9253),
    rotation=Rotation(x=0, y=0, z=248.74047228187703)
)

In [14]:
async def pick_up_from(target_position):
    waypoint_0 = deepcopy(target_position)
    waypoint_0.location = waypoint_0.location + Coordinate(0,0,100)

    await kx2.servo_gripper_open(27)

    await kx2.move_to_cartesian_position(
            waypoint_0,
            vel_pct=20,
            accel_pct=20
        )
    
    await kx2.move_to_cartesian_position(
            target_position,
            vel_pct=10,
            accel_pct=20
        )

    await kx2.servo_gripper_close()

    await kx2.move_to_cartesian_position(
        waypoint_0,
        vel_pct=20,
        accel_pct=20
    )

In [15]:
async def drop_to(target_position):
    waypoint_0 = deepcopy(target_position)
    waypoint_0.location = waypoint_0.location + Coordinate(0,0,100)

    await kx2.move_to_cartesian_position(
            waypoint_0,
            vel_pct=20,
            accel_pct=20
        )
    
    await kx2.move_to_cartesian_position(
            target_position,
            vel_pct=10,
            accel_pct=20
        )

    await kx2.servo_gripper_open(27)

    await kx2.move_to_cartesian_position(
        waypoint_0,
        vel_pct=20,
        accel_pct=20
    )

### Move Routines

In [24]:
await kx2.servo_gripper_close()

cmd {<KX2Axis.SERVO_GRIPPER: 6>: 0}
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 MS 1  False (1)
Servo Gripper Motor Status: 1
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)


In [25]:
await kx2.get_joint_position()

{<KX2Axis.SHOULDER: 1>: 21.086449364078195,
 <KX2Axis.Z: 2>: 19.831220308363864,
 <KX2Axis.ELBOW: 3>: 399.8096555997827,
 <KX2Axis.WRIST: 4>: 176.33056640625,
 <KX2Axis.SERVO_GRIPPER: 6>: 0.865508768245152}

In [27]:
await kx2.servo_gripper_open(1.0)

cmd {<KX2Axis.SERVO_GRIPPER: 6>: 1.0}


In [18]:
await kx2.move_to_cartesian_position(
        plate_on_byonoy_reader_location,
        vel_pct=20,
        accel_pct=20
    )

cmd {<KX2Axis.SHOULDER: 1>: 21.086449100755296, <KX2Axis.Z: 2>: 19.831, <KX2Axis.ELBOW: 3>: 399.8096881758738, <KX2Axis.WRIST: 4>: 176.3305666695729}


#### Byonoy

In [14]:
await pick_up_from(plate_on_byonoy_reader_location)

cmd {<KX2Axis.SERVO_GRIPPER: 6>: 27}
cmd {<KX2Axis.SHOULDER: 1>: 21.086449100755296, <KX2Axis.Z: 2>: 119.831, <KX2Axis.ELBOW: 3>: 399.8096881758738, <KX2Axis.WRIST: 4>: 176.3305666695729}
cmd {<KX2Axis.SHOULDER: 1>: 21.086449100755296, <KX2Axis.Z: 2>: 19.831, <KX2Axis.ELBOW: 3>: 399.8096881758738, <KX2Axis.WRIST: 4>: 176.3305666695729}
cmd {<KX2Axis.SERVO_GRIPPER: 6>: 0}
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 MS 1  False (1)
Servo Gripper Motor Status: 1
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)
cmd {<KX2Axis.SHOULDER: 1>: 21.086449100755296, <KX2Axis.Z: 2>: 119.831, <KX2Axis.ELBOW: 3>: 399.8096881758738, <KX2Axis.WRIST: 4>: 176.3305666695729}


In [19]:
await drop_to(plate_on_byonoy_reader_location)

cmd {<KX2Axis.SHOULDER: 1>: 21.086449100755296, <KX2Axis.Z: 2>: 119.831, <KX2Axis.ELBOW: 3>: 399.8096881758738, <KX2Axis.WRIST: 4>: 176.3305666695729}
cmd {<KX2Axis.SHOULDER: 1>: 21.086449100755296, <KX2Axis.Z: 2>: 19.831, <KX2Axis.ELBOW: 3>: 399.8096881758738, <KX2Axis.WRIST: 4>: 176.3305666695729}
cmd {<KX2Axis.SERVO_GRIPPER: 6>: 28}
cmd {<KX2Axis.SHOULDER: 1>: 21.086449100755296, <KX2Axis.Z: 2>: 119.831, <KX2Axis.ELBOW: 3>: 399.8096881758738, <KX2Axis.WRIST: 4>: 176.3305666695729}


In [25]:
await pick_up_from(byonoy_parking_location)

await drop_to(byonoy_reader_location)

cmd {<KX2Axis.SERVO_GRIPPER: 6>: 27}
cmd {<KX2Axis.SHOULDER: 1>: -0.8344501115604355, <KX2Axis.Z: 2>: 137.5711, <KX2Axis.ELBOW: 3>: 425.38009745465695, <KX2Axis.WRIST: 4>: 197.88574654463872}
cmd {<KX2Axis.SHOULDER: 1>: -0.8344501115604355, <KX2Axis.Z: 2>: 37.5711, <KX2Axis.ELBOW: 3>: 425.38009745465695, <KX2Axis.WRIST: 4>: 197.88574654463872}
cmd {<KX2Axis.SERVO_GRIPPER: 6>: 0}
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 MS 1  False (1)
Servo Gripper Motor Status: 1
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)
cmd {<KX2Axis.SHOULDER: 1>: -0.8344501115604355, <KX2Axis.Z: 2>: 137.5711, <KX2Axis.ELBOW: 3>: 425.38009745465695, <KX2Axis.WRIST: 4>: 197.88574654463872}
cmd {<KX2Axis.SHOULDER: 1>: 21.16743437808717, <KX2Axis.Z: 2>: 137.7

In [27]:
await pick_up_from(byonoy_reader_location)

await drop_to(byonoy_parking_location)

cmd {<KX2Axis.SERVO_GRIPPER: 6>: 27}
cmd {<KX2Axis.SHOULDER: 1>: 21.16743437808717, <KX2Axis.Z: 2>: 137.7131, <KX2Axis.ELBOW: 3>: 405.07788814062667, <KX2Axis.WRIST: 4>: 175.84716419001123}
cmd {<KX2Axis.SHOULDER: 1>: 21.16743437808717, <KX2Axis.Z: 2>: 37.7131, <KX2Axis.ELBOW: 3>: 405.07788814062667, <KX2Axis.WRIST: 4>: 175.84716419001123}
cmd {<KX2Axis.SERVO_GRIPPER: 6>: 0}
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 MS 1  False (1)
Servo Gripper Motor Status: 1
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)
cmd {<KX2Axis.SHOULDER: 1>: 21.16743437808717, <KX2Axis.Z: 2>: 137.7131, <KX2Axis.ELBOW: 3>: 405.07788814062667, <KX2Axis.WRIST: 4>: 175.84716419001123}
cmd {<KX2Axis.SHOULDER: 1>: -0.8344501115604355, <KX2Axis.Z: 2>: 137.5711,

In [28]:
await pick_up_from(plate_on_byonoy_reader_location)

await drop_to(plate_in_scilar_drawer_2_location)

cmd {<KX2Axis.SERVO_GRIPPER: 6>: 27}
cmd {<KX2Axis.SHOULDER: 1>: 21.086449100755296, <KX2Axis.Z: 2>: 119.831, <KX2Axis.ELBOW: 3>: 399.8096881758738, <KX2Axis.WRIST: 4>: 176.3305666695729}
cmd {<KX2Axis.SHOULDER: 1>: 21.086449100755296, <KX2Axis.Z: 2>: 19.831, <KX2Axis.ELBOW: 3>: 399.8096881758738, <KX2Axis.WRIST: 4>: 176.3305666695729}
cmd {<KX2Axis.SERVO_GRIPPER: 6>: 0}
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 MS 1  False (1)
Servo Gripper Motor Status: 1
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)
cmd {<KX2Axis.SHOULDER: 1>: 21.086449100755296, <KX2Axis.Z: 2>: 119.831, <KX2Axis.ELBOW: 3>: 399.8096881758738, <KX2Axis.WRIST: 4>: 176.3305666695729}
cmd {<KX2Axis.SHOULDER: 1>: -17.85376924580999, <KX2Axis.Z: 2>: 138.9253, <KX2Ax

In [29]:
await pick_up_from(plate_in_scilar_drawer_2_location)

await drop_to(plate_on_byonoy_reader_location)

cmd {<KX2Axis.SERVO_GRIPPER: 6>: 27}
cmd {<KX2Axis.SHOULDER: 1>: -17.85376924580999, <KX2Axis.Z: 2>: 138.9253, <KX2Axis.ELBOW: 3>: 172.80149536246222, <KX2Axis.WRIST: 4>: 266.594241527687}
cmd {<KX2Axis.SHOULDER: 1>: -17.85376924580999, <KX2Axis.Z: 2>: 38.9253, <KX2Axis.ELBOW: 3>: 172.80149536246222, <KX2Axis.WRIST: 4>: 266.594241527687}
cmd {<KX2Axis.SERVO_GRIPPER: 6>: 0}
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 MS 1  False (1)
Servo Gripper Motor Status: 1
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)
cmd {<KX2Axis.SHOULDER: 1>: -17.85376924580999, <KX2Axis.Z: 2>: 138.9253, <KX2Axis.ELBOW: 3>: 172.80149536246222, <KX2Axis.WRIST: 4>: 266.594241527687}
cmd {<KX2Axis.SHOULDER: 1>: 21.086449100755296, <KX2Axis.Z: 2>: 119.831, <KX2

In [16]:
# await pick_up_from(plate_on_byonoy_reader_location)

await kx2.move_to_cartesian_position(
        idle_location,
        vel_pct=25,
        accel_pct=20
    )

await drop_to(plate_in_pico_location)

cmd {<KX2Axis.SHOULDER: 1>: -0.8291222518411464, <KX2Axis.Z: 2>: 410.0812, <KX2Axis.ELBOW: 3>: 282.47135570157036, <KX2Axis.WRIST: 4>: 196.89697065508622}
cmd {<KX2Axis.SHOULDER: 1>: -26.498030413106118, <KX2Axis.Z: 2>: 323.1917, <KX2Axis.ELBOW: 3>: 498.09884312562644, <KX2Axis.WRIST: 4>: 227.92236362739618}
cmd {<KX2Axis.SHOULDER: 1>: -26.498030413106118, <KX2Axis.Z: 2>: 223.1917, <KX2Axis.ELBOW: 3>: 498.09884312562644, <KX2Axis.WRIST: 4>: 227.92236362739618}
cmd {<KX2Axis.SERVO_GRIPPER: 6>: 27}
cmd {<KX2Axis.SHOULDER: 1>: -26.498030413106118, <KX2Axis.Z: 2>: 323.1917, <KX2Axis.ELBOW: 3>: 498.09884312562644, <KX2Axis.WRIST: 4>: 227.92236362739618}


In [ ]:
awai

In [23]:
await kx2.servo_gripper_open(28)

await kx2.move_to_cartesian_position(
        waypoint_0,
        vel_pct=10,
        accel_pct=20
    )

cmd {<KX2Axis.SERVO_GRIPPER: 6>: 28}
cmd {<KX2Axis.SHOULDER: 1>: 21.16743437808717, <KX2Axis.Z: 2>: 137.7131, <KX2Axis.ELBOW: 3>: 405.07788814062667, <KX2Axis.WRIST: 4>: 175.84716419001123}


cmd {<KX2Axis.SHOULDER: 1>: -0.8344501115604355, <KX2Axis.Z: 2>: 137.5711, <KX2Axis.ELBOW: 3>: 425.38009745465695, <KX2Axis.WRIST: 4>: 197.88574654463872}


CanError: Timeout waiting for response to MS[0] from node 2

In [36]:

async def move_to_position(
    position: GripperPose,
    speed: float = 20.0,
    approach_speed: float = 10.00,
    acceleration: float = 20.0
):

    # Waypoints
    waypoint_0 = position
    waypoint_0.location + Coordinate(0,0,200)

    await kx2.move_to_cartesian_position(
        waypoint_0,
        vel_pct=speed,
        accel_pct=acceleration
    )

    # Approach
    await kx2.move_to_cartesian_position(
        position,
        vel_pct=10.0,
        accel_pct=acceleration
    )

    await kx2.servo_gripper_open(open_position=23.0)

    # Retract
    await kx2.move_to_cartesian_position(
        waypoint_0,
        vel_pct=speed,
        accel_pct=acceleration
    )

    # Slightly close gripper
    await kx2.servo_gripper_open(open_position=20.0)


In [37]:
await move_to_position(plate_on_byonoy_reader_location)

cmd {<KX2Axis.SHOULDER: 1>: 21.086449100755296, <KX2Axis.Z: 2>: 19.831, <KX2Axis.ELBOW: 3>: 399.8096881758738, <KX2Axis.WRIST: 4>: 176.3305666695729}
ax 2 target 19.831 low 0.0 high 750.0
ax 3 target 399.8096881758738 low 0.0 high 525.0


CanError: Motor move error on node 3: Position tracking error

In [ ]:
await kx2.activate_free_mode()

In [28]:
await kx2.servo_gripper_close()

cmd {<KX2Axis.SERVO_GRIPPER: 6>: 0}
ax 6 target 0 low 0.0 high 26.149999618530273
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 MS 1  False (1)
Servo Gripper Motor Status: 1
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)


In [56]:
await kx2.deactivate_free_mode()

CanError: Timeout waiting for response to MO[0] from node 1

In [9]:
kx2.min_travel_ax

{1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 6: 0.0}

In [13]:
kx2.unlimited_travel_ax

{1: True, 2: False, 3: False, 4: True, 6: False}

In [43]:
await kx2.move_to_cartesian_position(
    GripperPose(
    location=Coordinate(x=-170.3944, y=0.9451, z=650),
    rotation=Rotation(x=0, y=0, z=0)
),
    vel_pct=20,
    accel_pct=20
)

cmd {<KX2Axis.SHOULDER: 1>: 89.68220970916462, <KX2Axis.Z: 2>: 650, <KX2Axis.ELBOW: 3>: 18.741022769126204, <KX2Axis.WRIST: 4>: 270.3177902908354}
ax 2 target 650 low 0.0 high 750.0
ax 3 target 18.741022769126204 low 0.0 high 525.0


In [55]:
await kx2.get_cartesian_position()

CanError: Timeout waiting for response to PU[0] from node 1

In [5]:
await kx2.move_to_cartesian_position(
    GripperPose(
        location=Coordinate(x=-26.9701, y=160.9649, z=04.6119),
        rotation=Rotation(x=0, y=0, z=-171.05957734864037)
    ),
    vel_pct=20,
    accel_pct=20
)

ValueError: Axis <KX2Axis.ELBOW: 3> below min travel

cmd {<KX2Axis.SERVO_GRIPPER: 6>: 23.0}
ax 6 target 23.0 low 0.0 high 26.149999618530273


In [46]:
await kx2.servo_gripper_close()

cmd {<KX2Axis.SERVO_GRIPPER: 6>: 0}
ax 6 target 0 low 0.0 high 26.149999618530273
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 MS 1  False (1)
Servo Gripper Motor Status: 1
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 CL 1  False (1)
node_id not int: 6 <enum 'KX2Axis'>
motor send command 6 IQ 0  False (1)


In [48]:
await kx2.get_joint_position()

{<KX2Axis.SHOULDER: 1>: 359.2025901788258,
 <KX2Axis.Z: 2>: 82.01458522649091,
 <KX2Axis.ELBOW: 3>: 427.09553382341323,
 <KX2Axis.WRIST: 4>: 18.1494140625,
 <KX2Axis.SERVO_GRIPPER: 6>: 7.635326716308593}

In [ ]:
await kx2.servo_gripper_open(open_position=10.0)

In [ ]:
home_position =  await kx2.get_cartesian_position()

home_position

In [ ]:
byonoy_dector_unit_location = await kx2.get_cartesian_position()
byonoy_dector_unit_location

In [ ]:
byonoy_dector_unit_location = GripperPose(location=Coordinate(x=-195.1244, y=518.7307, z=37.6328), rotation=Rotation(x=0, y=0, z=-161.802826074163))

In [ ]:
waypoint_z_0 = GripperPose(location=Coordinate(x=-195.1244, y=518.7307, z=37.6328 + 100), rotation=Rotation(x=0, y=0, z=-161.802826074163))

In [ ]:
await kx2.move_to_cartesian_position(
    waypoint_z_0,
    vel_pct=20,
    accel_pct=20
)

In [ ]:
await kx2.move_to_cartesian_position(
    byonoy_dector_unit_location,
    vel_pct=20,
    accel_pct=20
)

In [ ]:
kx2.servo_gripper_closed_position = 8
await kx2.servo_gripper_close()

In [ ]:
gp = GripperPose(location=Coordinate(x=222.8748, y=185.6677, z=520.8049), rotation=Rotation(x=0, y=0, z=-122.82045997896907))
await kx2.move_to_cartesian_position(gp, vel_pct=20, accel_pct=20)

In [ ]:
drawer4 = GripperPose(location=Coordinate(x=101.001, y=296.6519, z=73.1333 + 47), rotation=Rotation(x=0, y=0, z=271.01718397191803))
await kx2.move_to_cartesian_position(drawer4, vel_pct=10, accel_pct=10)

In [ ]:
top = GripperPose(location=Coordinate(x=101.001, y=296.6519, z=300), rotation=Rotation(x=0, y=0, z=271.01718397191803))
await kx2.move_to_cartesian_position(top)

In [ ]:
await scila.open_drawer(4)

In [ ]:
await kx2.move_to_cartesian_position(drawer4, vel_pct=10, accel_pct=10)

In [ ]:
await kx2.servo_gripper_close()

In [ ]:
await kx2.move_to_cartesian_position(top, vel_pct=10, accel_pct=10)

In [ ]:
await scila.close_drawer(4)

In [ ]:
await scila.open_drawer(3)

In [ ]:
drawer3 = GripperPose(location=Coordinate(x=101.001, y=296.6519, z=73.1333), rotation=Rotation(x=0, y=0, z=271.01718397191803))
await kx2.move_to_cartesian_position(drawer3, vel_pct=10, accel_pct=10)

In [ ]:
await kx2.servo_gripper_open()

In [ ]:
await kx2.move_to_cartesian_position(top, vel_pct=10, accel_pct=10)

In [ ]:
await scila.close_drawer(3)